<a href="https://colab.research.google.com/github/douglaslpo/DC-ETL-Treatments---DCETLT/blob/main/ETL_Siga_Empreendimentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from geopy.geocoders import Nominatim
import pandas as pd
import time  # Importação da biblioteca time

# Configurar o geolocalizador
geolocator = Nominatim(user_agent="geoapi")

# Carregar o arquivo CSV
arquivo_csv = 'siga-empreendimentos-geracao.csv'  # Substitua pelo nome do arquivo
df = pd.read_csv(arquivo_csv, encoding='utf-8')

# Filtrar as linhas que contêm "100% para"
df['Percentual Participação'] = None  # Criar a nova coluna inicialmente vazia
filtro = df['DscPropriRegimePariticipacao'].str.contains("100% para", na=False)  # Substitua 'sua_coluna' pelo nome correto da coluna
df.loc[filtro, 'Percentual Participação'] = '100%'

# Remover "100% para" da coluna original
df.loc[filtro, 'DscPropriRegimePariticipacao'] = df.loc[filtro, 'DscPropriRegimePariticipacao'].str.replace("100% para", "", regex=False).str.strip()

# Substituir vírgulas por pontos e converter as colunas para numérico
df['NumCoordNEmpreendimento'] = df['NumCoordNEmpreendimento'].astype(str).str.replace(',', '.').astype(float)
df['NumCoordEEmpreendimento'] = df['NumCoordEEmpreendimento'].astype(str).str.replace(',', '.').astype(float)

# Função para buscar dados geográficos de múltiplas coordenadas
def buscar_dados_geograficos(df):
    for index, row in df.iterrows():
        try:
            lat, lon = row['NumCoordNEmpreendimento'], row['NumCoordEEmpreendimento']
            location = geolocator.reverse((lat, lon), exactly_one=True, timeout=10)
            if location and 'address' in location.raw:
                address = location.raw['address']
                municipio = address.get('city', address.get('town', address.get('village', 'Município não encontrado')))
                estado = address.get('state', 'Estado não encontrada')
            else:
                municipio, estado = 'Município não encontrado', 'Estado não encontrada'

            # Adicionar os valores ao DataFrame
            df.at[index, 'MUNICIPIO'] = municipio
            df.at[index, 'ESTADO'] = estado

            # Pausa para evitar sobrecarga na API
            time.sleep(1)

            # Salvar progresso a cada 1000 registros processados
            if index % 1000 == 0:
                df.to_csv('municipios_parcial.csv', index=False, encoding='utf-8')
                print(f"Progresso salvo até o índice {index}")
        except Exception as e:
            df.at[index, 'MUNICIPIO'] = f"Erro: {e}"
            df.at[index, 'ESTADO'] = f"Erro: {e}"
            print(f"Erro ao processar índice {index}: {e}")

# Chamar a função para processar as coordenadas
buscar_dados_geograficos(df)

# Salvar o resultado final
df.to_csv('municipios_uf_resultado.csv', index=False, encoding='utf-8')

print("Processamento concluído. Resultado salvo como 'municipios_uf_resultado.csv'.")


Progresso salvo até o índice 0
Progresso salvo até o índice 1000
Progresso salvo até o índice 2000
Progresso salvo até o índice 3000
Progresso salvo até o índice 4000
Progresso salvo até o índice 5000
Progresso salvo até o índice 6000
Progresso salvo até o índice 7000
Progresso salvo até o índice 8000
Progresso salvo até o índice 9000
Progresso salvo até o índice 10000
Progresso salvo até o índice 11000
